In [ ]:
import pandas as pd
import numpy as np
import re
import datetime as time
from collections import Counter
from mlxtend.frequent_patterns import apriori, association_rules
#pd.set_option('display.max_columns', None)
df = pd.read_parquet('scontrini_2.parquet',engine='pyarrow')
df = df[(~df.reparto.isin(["95","80","85"]))&(df.tipo=="")]
df

In [ ]:
def hot_encode(x):
    if(x==0):
        return False
    else:
        return True

In [ ]:
transazioni=df[['cassa','transazione','data_reg','descrizione','reparto','qta_riga']]

Frequent set dei reparti periodo settebre-gennaio

In [ ]:
frq_items_reparti={}
for reparto in df.reparto.unique():
    
    basket_rep = transazioni[transazioni.reparto==reparto]\
    .groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga']\
    .sum()\
    .unstack(fill_value=0)\
    .reset_index()\
    .drop(['cassa', 'transazione','data_reg'], axis=1)\
    .applymap(hot_encode)

    frq_items=apriori(basket_rep, min_support = 0.001, use_colnames = True)
    frq_items_reparti[reparto]=frq_items

In [ ]:
for rep in frq_items_reparti.keys():
    print("\n" +"="*50)
    print(rep)
    print("-"*50)
    print(frq_items_reparti[rep].sort_values(by=["support"],ascending=False).head())

In [ ]:
association_rules_reparti={}
for rep in frq_items_reparti.keys():
    print("\n" +"="*100)
    print("Reparto: "+rep)
    print("-"*100)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(association_rules(frq_items_reparti[rep], metric ="confidence", min_threshold = 0.01).sort_values(by=['confidence'], ascending=False).head(10))